In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load
%load_ext cudf.pandas
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 500)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import warnings
warnings.filterwarnings("ignore")

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/playground-series-s5e4/sample_submission.csv
/kaggle/input/playground-series-s5e4/train.csv
/kaggle/input/playground-series-s5e4/test.csv


In [2]:
train = pd.read_csv('/kaggle/input/playground-series-s5e4/train.csv')
test = pd.read_csv('/kaggle/input/playground-series-s5e4/test.csv')
all_data = pd.concat([train, test], axis=0)

## Data Understanding

In [3]:
train.sample(10)

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes
747338,747338,Business Insights,Episode 59,76.24,Business,44.69,Thursday,Morning,<NA>,0.0,Neutral,72.46000
124266,124266,Business Briefs,Episode 84,119.14,Business,42.62,Monday,Night,18.21,1.0,Neutral,75.13915
130310,130310,Global News,Episode 77,47.02,News,22.20,Monday,Afternoon,<NA>,2.0,Negative,36.96629
677238,677238,Music Matters,Episode 69,65.76,Music,42.20,Thursday,Afternoon,17.34,1.0,Negative,46.99398
29268,29268,True Crime Stories,Episode 42,<NA>,True Crime,66.17,Wednesday,Afternoon,8.12,0.0,Neutral,40.72448
436533,436533,Tech Talks,Episode 99,30.36,Technology,26.70,Thursday,Afternoon,54.22,0.0,Positive,10.61403
574738,574738,Wellness Wave,Episode 71,33.47,Health,96.67,Wednesday,Night,94.98,0.0,Neutral,24.39751
536234,536234,Funny Folks,Episode 11,96.37,Comedy,61.56,Saturday,Afternoon,80.28,1.0,Positive,74.05417
362454,362454,Home & Living,Episode 85,44.18,Lifestyle,31.50,Saturday,Night,34.67,0.0,Neutral,24.25950
279981,279981,Mind & Body,Episode 81,74.81,Health,66.65,Sunday,Evening,30.72,3.0,Positive,45.38866


In [4]:
train.describe()

,id,Episode_Length_minutes,Host_Popularity_percentage,Guest_Popularity_percentage,Number_of_Ads,Listening_Time_minutes
count,750000.000000,662907.000000,750000.000000,603970.000000,749999.000000,750000.000000
mean,374999.500000,64.504738,59.859901,52.236449,1.348855,45.437406
std,216506.495284,32.969603,22.873098,28.451241,1.151130,27.138306
min,0.000000,0.000000,1.300000,0.000000,0.000000,0.000000
25%,187499.750000,35.730000,39.410000,28.380000,0.000000,23.178350
50%,374999.500000,63.840000,60.050000,53.580000,1.000000,43.379460
75%,562499.250000,94.070000,79.530000,76.600000,2.000000,64.811580
max,749999.000000,325.240000,119.460000,119.910000,103.910000,119.970000


In [5]:
train.shape

(750000, 12)

In [6]:
test.shape

(250000, 11)

In [7]:
train.info()

<class 'cudf.core.dataframe.DataFrame'>
RangeIndex: 750000 entries, 0 to 749999
Data columns (total 12 columns):
 #   Column                       Non-Null Count   Dtype
---  ------                       --------------   -----
 0   id                           750000 non-null  int64
 1   Podcast_Name                 750000 non-null  object
 2   Episode_Title                750000 non-null  object
 3   Episode_Length_minutes       662907 non-null  float64
 4   Genre                        750000 non-null  object
 5   Host_Popularity_percentage   750000 non-null  float64
 6   Publication_Day              750000 non-null  object
 7   Publication_Time             750000 non-null  object
 8   Guest_Popularity_percentage  603970 non-null  float64
 9   Number_of_Ads                749999 non-null  float64
 10  Episode_Sentiment            750000 non-null  object
 11  Listening_Time_minutes       750000 non-null  float64
dtypes: float64(5), int64(1), object(6)
memory usage: 88.8+ MB


In [8]:
train.isnull().sum()

id                                  0
Podcast_Name                        0
Episode_Title                       0
Episode_Length_minutes          87093
Genre                               0
Host_Popularity_percentage          0
Publication_Day                     0
Publication_Time                    0
Guest_Popularity_percentage    146030
Number_of_Ads                       1
Episode_Sentiment                   0
Listening_Time_minutes              0
dtype: int64

In [9]:
test.isnull().sum()

id                                 0
Podcast_Name                       0
Episode_Title                      0
Episode_Length_minutes         28736
Genre                              0
Host_Popularity_percentage         0
Publication_Day                    0
Publication_Time                   0
Guest_Popularity_percentage    48832
Number_of_Ads                      0
Episode_Sentiment                  0
dtype: int64

## Data Preparation

In [10]:
from sklearn.impute import SimpleImputer

impute = SimpleImputer(strategy='median')
num_cols = test.select_dtypes(include=['float64']).columns
train[num_cols] = impute.fit_transform(train[num_cols])
test[num_cols] = impute.fit_transform(test[num_cols])

In [11]:
train.columns

Index(['id', 'Podcast_Name', 'Episode_Title', 'Episode_Length_minutes',
       'Genre', 'Host_Popularity_percentage', 'Publication_Day',
       'Publication_Time', 'Guest_Popularity_percentage', 'Number_of_Ads',
       'Episode_Sentiment', 'Listening_Time_minutes'],
      dtype='object')

## Feature engineering

In [12]:
bins = [0, 81, 162, 243, 326]
labels = ['short', 'medium', 'large', 'very large']
train['length_binning'] = pd.cut(train['Episode_Length_minutes'], bins=bins, labels=labels, include_lowest=True)
test['length_binning'] = pd.cut(test['Episode_Length_minutes'], bins=bins, labels=labels, include_lowest=True)

In [13]:
def add_features(df):
    df['Guest_Popularity_percentage'] =  df['Guest_Popularity_percentage'].clip(upper=100)
    df['Host_Popularity_percentage'] =  df['Host_Popularity_percentage'].clip(upper=100)
    df['Number_of_Ads'] = df['Number_of_Ads'].astype(int)
    
    df['IsWeekend'] = df['Publication_Day'].isin(['Saturday', 'Sunday']).astype(int)
    
    sentiments = {'Positive': 1, 'Neutral': 0, 'Negative': -1}
    df['Episode_Sentiment'] = df['Episode_Sentiment'].map(sentiments)
    
    df['Guest_Popularity_lengthmin'] =  df['Guest_Popularity_percentage'] * df['Episode_Length_minutes']
    df['Host_Popularity_lengthmin'] =  df['Host_Popularity_percentage'] * df['Episode_Length_minutes']
    df['Average_popularity'] = (df['Host_Popularity_percentage'] + df['Guest_Popularity_percentage']) / 2
    df['minutes_per_ads'] = df['Episode_Length_minutes'] / (df['Number_of_Ads'] + 1) 
    
    return df 

In [14]:
cat_cols = ['Podcast_Name', 'Genre', 'Publication_Day', 'Publication_Time']
for colname in cat_cols:
    train[colname] = train[colname].astype('category')
    test[colname] = test[colname].astype('category')

## Count Encoding for high cardinality columns

In [15]:
train.head()

,id,Podcast_Name,Episode_Title,Episode_Length_minutes,Genre,Host_Popularity_percentage,Publication_Day,Publication_Time,Guest_Popularity_percentage,Number_of_Ads,Episode_Sentiment,Listening_Time_minutes,length_binning
0,0,Mystery Matters,Episode 98,63.84,True Crime,74.81,Thursday,Night,53.58,0.0,Positive,31.41998,short
1,1,Joke Junction,Episode 26,119.80,Comedy,66.95,Saturday,Afternoon,75.95,2.0,Negative,88.01241,medium
2,2,Study Sessions,Episode 16,73.90,Education,69.97,Tuesday,Evening,8.97,0.0,Negative,44.92531,short
3,3,Digital Digest,Episode 45,67.17,Technology,57.22,Monday,Morning,78.70,2.0,Positive,46.27824,short
4,4,Mind & Body,Episode 86,110.51,Health,80.07,Monday,Afternoon,58.68,3.0,Neutral,75.61031,medium


In [16]:
from category_encoders import CountEncoder

ce = CountEncoder()
count_category = ['Podcast_Name', 'Genre', 'Episode_Title', 'length_binning']
train[count_category] = ce.fit_transform(train[count_category])
test[count_category] = ce.transform(test[count_category])

In [17]:
from sklearn.preprocessing import LabelEncoder

cat_cols = ['Publication_Day', 'Publication_Time']
for colname in cat_cols:
    le = LabelEncoder()
    train[colname] = le.fit_transform(train[colname])
    test[colname] = le.transform(test[colname])

In [18]:
cat_features = ['Publication_Day', 'Publication_Time', 'Podcast_Name', 'Genre', 'Episode_Sentiment', 'Episode_Title']
def combo_1(df):
    for i, col1 in enumerate(cat_features):
        for col2 in (cat_features[i+1:]):
            new_col = f'{col1}_{col2}'
            df[new_col] = df[col1] * df[col2]
    return df

## Splitting the train data by KFold

In [19]:
from sklearn.model_selection import KFold

X = train.copy()
y = X.pop('Listening_Time_minutes')
X = add_features(X)
X = combo_1(X)

X_test = test.copy()
X_test = add_features(X_test)
X_test = combo_1(X_test)

kf = KFold(n_splits=10, shuffle=True, random_state=600)

for train_index, valid_index in kf.split(X, y):
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]  

## Target encoding

In [20]:
stats = ['mean', 'median', 'std', 'skew', 'count', 'max', 'min', 'nunique']

train_with_target = X_train.copy()
train_with_target['Listening_Time_minutes'] = y_train

grouped_stats = train_with_target.groupby(['Episode_Length_minutes'])['Listening_Time_minutes'].agg(stats).fillna(0.0)
grouped_stats.columns = [f'TE1_WC_{stat}' for stat in stats]

grouped_stats_full = train.groupby(['Episode_Length_minutes'])['Listening_Time_minutes'].agg(stats).fillna(0.0)
grouped_stats_full.columns = [f'TE1_WC_{stat}' for stat in stats]

X_train = X_train.merge(grouped_stats, on='Episode_Length_minutes', how='left')
X_valid = X_valid.merge(grouped_stats_full, on='Episode_Length_minutes', how='left')
X_test = X_test.merge(grouped_stats_full, on='Episode_Length_minutes', how='left')

In [21]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error

xgb = XGBRegressor(
    n_estimators=386, 
    learning_rate=0.07, 
    max_depth=11, 
    random_state=600, 
    reg_lambda=8, 
    reg_alpha= 7, 
    device='cuda',
    verbose=0, 
    eval_metric='rmse'
    ).fit(X_train, y_train)
pred_xgb = xgb.predict(X_valid)
print(f'RMSE: {np.sqrt(mean_squared_error(y_valid, pred_xgb))}')

RMSE: 12.680399061284708


## Final Submission

In [22]:
test_preds = xgb.predict(X_test)

output = pd.DataFrame({
    'id': test['id'],
    'Listening_Time_minutes': test_preds
})
output.to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!
